<a href="https://colab.research.google.com/github/Rohanshetty28/Warehouse/blob/main/Personal%20Finance%20Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
def install_requirements():
    """Install required packages automatically"""
    packages = ['gradio', 'transformers>=4.35.0', 'torch', 'pandas', 'accelerate', 'huggingface-hub', 'numpy']

In [7]:
# Personal Finance Chatbot with IBM Granite 3.3
# Install required packages first

import subprocess
import sys
import os

def install_requirements():
    """Install required packages"""
    packages = [
        'gradio',
        'transformers>=4.35.0',
        'torch',
        'pandas',
        'accelerate',
        'huggingface-hub',
        'numpy'
    ]

    for package in packages:
        try:
            __import__(package.split('>=')[0].split('==')[0])
            print(f"✅ {package} already installed")
        except ImportError:
            print(f"📦 Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    print("🎉 All packages installed successfully!")

# Install requirements
try:
    install_requirements()
except Exception as e:
    print(f"⚠️ Error installing packages: {e}")
    print("Please run: pip install gradio transformers torch pandas accelerate huggingface-hub numpy")

# Now import the packages
try:
    import gradio as gr
    import json
    import re
    from typing import Dict, List, Tuple, Optional
    from dataclasses import dataclass, asdict
    from datetime import datetime
    import pandas as pd
    from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
    import torch
    import warnings
    warnings.filterwarnings('ignore')

except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please restart your kernel/runtime after package installation")
    sys.exit(1)

# Configuration
@dataclass
class UserProfile:
    name: str = ""
    age: int = 0
    occupation: str = ""
    income: float = 0.0
    user_type: str = "professional"  # "student" or "professional"
    risk_tolerance: str = "moderate"  # "conservative", "moderate", "aggressive"
    financial_goals: List[str] = None

    def __post_init__(self):
        if self.financial_goals is None:
            self.financial_goals = []

class FinanceChatbot:
    def __init__(self, hf_token: str):
        """Initialize the chatbot with IBM Granite model"""
        self.hf_token = hf_token
        self.model_name = "ibm-granite/granite-3.3-2b-instruct"
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = None
        self.tokenizer = None
        self.generator = None

        # Initialize model
        self._load_model()

        self.user_profiles = {}
        self.conversation_history = {}

        # Financial knowledge base
        self.financial_knowledge = {
            "investment_types": {
                "stocks": "Shares of ownership in companies with potential for growth and dividends",
                "bonds": "Fixed-income securities that pay regular interest",
                "mutual_funds": "Diversified portfolios managed by professionals",
                "etfs": "Exchange-traded funds that track market indices",
                "real_estate": "Property investments for rental income and appreciation"
            },
            "tax_tips": {
                "deductions": ["401k contributions", "Mortgage interest", "Charitable donations", "Student loan interest"],
                "strategies": ["Tax-loss harvesting", "Roth IRA conversions", "HSA contributions", "Tax-deferred accounts"]
            },
            "budgeting_rules": {
                "50_30_20": "50% needs, 30% wants, 20% savings",
                "pay_yourself_first": "Save before spending on discretionary items",
                "emergency_fund": "3-6 months of expenses in easily accessible account"
            }
        }

    def _load_model(self):
        """Load the IBM Granite model with error handling"""
        try:
            print(f"🔄 Loading IBM Granite model on {self.device}...")

            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                token=self.hf_token,
                trust_remote_code=True
            )

            # Add pad token if missing
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Load model with appropriate settings
            model_kwargs = {
                "token": self.hf_token,
                "trust_remote_code": True,
                "torch_dtype": torch.float16 if torch.cuda.is_available() else torch.float32,
                "low_cpu_mem_usage": True
            }

            if torch.cuda.is_available():
                model_kwargs["device_map"] = "auto"

            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                **model_kwargs
            )

            # Create pipeline
            self.generator = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                device=0 if torch.cuda.is_available() else -1,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
            )

            print("✅ IBM Granite model loaded successfully!")

        except Exception as e:
            print(f"❌ Error loading model: {str(e)}")
            # Fallback to a basic response system
            self.model = None
            self.tokenizer = None
            self.generator = None
            print("⚠️ Using fallback response system")

    def get_user_type_prompt_style(self, user_type: str) -> str:
        """Get appropriate communication style based on user type"""
        if user_type == "student":
            return """You are a friendly, encouraging financial advisor speaking to a college student.
            Use simple language, relatable examples, and focus on building good financial habits.
            Be supportive and understanding of limited budgets. Include practical tips for students."""
        else:
            return """You are a professional financial advisor speaking to a working professional.
            Use more sophisticated financial terminology when appropriate, provide detailed analysis,
            and focus on wealth building strategies, tax optimization, and investment planning."""

    def create_financial_prompt(self, query: str, user_profile: UserProfile, context: str = "") -> str:
        """Create a comprehensive prompt for financial guidance"""
        style_guide = self.get_user_type_prompt_style(user_profile.user_type)

        prompt = f"""<|system|>
{style_guide}

You are an expert personal finance advisor with deep knowledge of:
- Investment strategies and portfolio management
- Tax planning and optimization
- Budgeting and expense management
- Savings strategies
- Financial planning for different life stages

User Profile:
- Type: {user_profile.user_type}
- Age: {user_profile.age}
- Occupation: {user_profile.occupation}
- Income: ${user_profile.income:,.2f}
- Risk Tolerance: {user_profile.risk_tolerance}
- Goals: {', '.join(user_profile.financial_goals) if user_profile.financial_goals else 'Not specified'}

Context: {context}

Provide specific, actionable financial advice. Include relevant calculations when helpful.
<|user|>
{query}
<|assistant|>
"""
        return prompt

    def get_fallback_response(self, query: str, user_profile: UserProfile, query_type: str = "general") -> str:
        """Provide fallback responses when model is unavailable"""
        query_lower = query.lower()

        # Investment advice
        if query_type == "investment" or any(word in query_lower for word in ['invest', 'stock', 'portfolio']):
            if user_profile.user_type == "student":
                return f"""Hi {user_profile.name}! As a student, here's my investment advice:

💡 **Start Small & Simple:**
- Begin with low-cost index funds or ETFs
- Consider apps like Robinhood or Fidelity for $0 commissions
- Start with just $25-50/month if that's what you can afford

📚 **Learning First:**
- Read "The Bogleheads' Guide to Investing"
- Understand the basics before investing
- Focus on long-term growth, not quick gains

🎯 **Student-Specific Tips:**
- Max out any employer 401(k) match if you have a part-time job
- Consider a Roth IRA - great for students in low tax brackets
- Avoid individual stocks until you have emergency savings

Remember: Time is your biggest advantage as a young investor!"""

            else:
                return f"""Hello {user_profile.name}! Based on your {user_profile.risk_tolerance} risk tolerance:

📊 **Portfolio Allocation Suggestion:**
- Conservative: 60% bonds, 40% stocks
- Moderate: 70% stocks, 30% bonds
- Aggressive: 85% stocks, 15% bonds

💼 **Professional Investment Strategy:**
- Max out 401(k) especially with employer match
- Consider target-date funds for simplicity
- Diversify across domestic/international markets
- Rebalance annually

🎯 **Income-Based Recommendations:**
- With ${user_profile.income:,.0f}/month income, aim to invest 15-20%
- Use tax-advantaged accounts first (401k, IRA)
- Emergency fund should be 3-6 months expenses before aggressive investing"""

        # Tax advice
        elif query_type == "tax" or any(word in query_lower for word in ['tax', 'deduction', 'irs']):
            return f"""Tax Planning Advice for {user_profile.name}:

💰 **Common Deductions:**
- 401(k) contributions (up to $23,000 in 2024)
- Student loan interest (up to $2,500)
- Mortgage interest
- State and local taxes (SALT) up to $10,000

📋 **Strategies for Your Income Level:**
- Income: ${user_profile.income:,.0f}/month (${user_profile.income*12:,.0f}/year)
- Consider Roth vs Traditional IRA based on current tax bracket
- HSA contributions are triple tax-advantaged
- Tax-loss harvesting for investment accounts

⚠️ **Important Note:**
Always consult a tax professional for personalized advice, especially for complex situations."""

        # Savings advice
        elif query_type == "savings" or any(word in query_lower for word in ['save', 'budget', 'emergency']):
            return f"""Savings Strategy for {user_profile.name}:

🎯 **Emergency Fund Priority:**
- Target: 3-6 months of expenses
- Keep in high-yield savings account
- Current target: ${user_profile.income*3:,.0f} - ${user_profile.income*6:,.0f}

📊 **50/30/20 Budget Rule:**
- 50% Needs (housing, utilities, food)
- 30% Wants (entertainment, dining out)
- 20% Savings & debt payments

💡 **Savings Tips:**
- Automate your savings
- Pay yourself first
- Use separate accounts for different goals
- Track spending with apps like Mint or YNAB

🚀 **Monthly Savings Goal:**
Based on your income, aim for ${user_profile.income*0.20:,.0f}/month in savings."""

        # General advice
        else:
            return f"""Hi {user_profile.name}! I'm here to help with your financial questions.

Based on your profile:
- Type: {user_profile.user_type.title()}
- Income: ${user_profile.income:,.0f}/month
- Risk Tolerance: {user_profile.risk_tolerance.title()}
- Goals: {', '.join(user_profile.financial_goals) if user_profile.financial_goals else 'Building financial wellness'}

I can help you with:
💰 Investment strategies and portfolio advice
📊 Tax planning and deduction optimization
💳 Budgeting and expense management
🎯 Savings strategies and emergency funds
📈 Financial planning for your life stage

Please ask me specific questions about any of these topics!"""

    def generate_response(self, prompt: str, max_length: int = 512) -> str:
        """Generate response using IBM Granite model or fallback"""
        if self.generator is None:
            return "I'm currently using a simplified response system. Please ask your financial question and I'll provide helpful guidance!"

        try:
            response = self.generator(
                prompt,
                max_new_tokens=max_length,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=self.tokenizer.eos_token_id,
                return_full_text=False
            )

            generated_text = response[0]['generated_text'].strip()

            # Clean up the response
            if '<|assistant|>' in generated_text:
                generated_text = generated_text.split('<|assistant|>')[-1].strip()

            return generated_text

        except Exception as e:
            print(f"⚠️ Model generation error: {e}")
            return "I encountered an error with the AI model. Let me provide some general financial guidance instead!"

    def analyze_spending_patterns(self, expenses: Dict[str, float], income: float) -> Dict:
        """Analyze spending patterns and provide insights"""
        total_expenses = sum(expenses.values())
        savings_rate = (income - total_expenses) / income * 100 if income > 0 else 0

        # Calculate percentages
        expense_percentages = {category: (amount / income * 100) if income > 0 else 0
                             for category, amount in expenses.items()}

        insights = []
        recommendations = []

        # Analysis based on 50/30/20 rule
        needs = ["housing", "utilities", "groceries", "transportation"]
        wants = ["entertainment", "dining_out", "shopping"]

        needs_total = sum(expenses.get(cat, 0) for cat in needs)
        wants_total = sum(expenses.get(cat, 0) for cat in wants)

        needs_percentage = (needs_total / income * 100) if income > 0 else 0
        wants_percentage = (wants_total / income * 100) if income > 0 else 0

        if needs_percentage > 50:
            insights.append(f"Essential expenses ({needs_percentage:.1f}%) exceed recommended 50%")
            recommendations.append("Look for ways to reduce housing or transportation costs")

        if wants_percentage > 30:
            insights.append(f"Discretionary spending ({wants_percentage:.1f}%) exceeds recommended 30%")
            recommendations.append("Consider reducing entertainment and dining expenses")

        if savings_rate < 20:
            insights.append(f"Savings rate ({savings_rate:.1f}%) is below recommended 20%")
            recommendations.append("Increase savings by optimizing expenses")
        elif savings_rate > 20:
            insights.append(f"Great job! Savings rate ({savings_rate:.1f}%) exceeds 20% target")

        return {
            "total_expenses": total_expenses,
            "savings_rate": savings_rate,
            "expense_percentages": expense_percentages,
            "insights": insights,
            "recommendations": recommendations,
            "needs_percentage": needs_percentage,
            "wants_percentage": wants_percentage
        }

    def generate_budget_summary(self, income: float, expenses: Dict[str, float],
                              user_profile: UserProfile) -> str:
        """Generate budget summary with or without AI model"""
        analysis = self.analyze_spending_patterns(expenses, income)

        if self.generator is not None:
            # Use AI model
            context = f"""
            Budget Analysis:
            - Monthly Income: ${income:,.2f}
            - Total Expenses: ${analysis['total_expenses']:,.2f}
            - Savings Rate: {analysis['savings_rate']:.1f}%
            - Needs: {analysis['needs_percentage']:.1f}% of income
            - Wants: {analysis['wants_percentage']:.1f}% of income
            """

            query = "Please create a comprehensive budget summary and provide actionable recommendations."
            prompt = self.create_financial_prompt(query, user_profile, context)
            return self.generate_response(prompt, max_length=400)

        else:
            # Fallback budget analysis
            status = "🔴 Needs Improvement" if analysis['savings_rate'] < 10 else "🟡 Fair" if analysis['savings_rate'] < 20 else "🟢 Excellent"

            return f"""**Budget Health Status: {status}**

**Monthly Overview:**
- Income: ${income:,.2f}
- Expenses: ${analysis['total_expenses']:,.2f}
- Savings: ${income - analysis['total_expenses']:,.2f}
- Savings Rate: {analysis['savings_rate']:.1f}%

**50/30/20 Rule Analysis:**
- Needs: {analysis['needs_percentage']:.1f}% (Target: 50%)
- Wants: {analysis['wants_percentage']:.1f}% (Target: 30%)
- Savings: {analysis['savings_rate']:.1f}% (Target: 20%)

**Key Insights:**
{chr(10).join(f'• {insight}' for insight in analysis['insights'])}

**Action Items:**
{chr(10).join(f'• {rec}' for rec in analysis['recommendations'])}
"""

    def get_investment_advice(self, query: str, user_profile: UserProfile) -> str:
        """Provide investment guidance"""
        if self.generator is not None:
            context = f"Risk Tolerance: {user_profile.risk_tolerance}, Investment Types: {self.financial_knowledge['investment_types']}"
            prompt = self.create_financial_prompt(query, user_profile, context)
            return self.generate_response(prompt)
        else:
            return self.get_fallback_response(query, user_profile, "investment")

    def get_tax_advice(self, query: str, user_profile: UserProfile) -> str:
        """Provide tax planning guidance"""
        if self.generator is not None:
            context = f"Tax strategies and deductions for income level: ${user_profile.income:,.2f}"
            prompt = self.create_financial_prompt(query, user_profile, context)
            return self.generate_response(prompt)
        else:
            return self.get_fallback_response(query, user_profile, "tax")

    def get_savings_advice(self, query: str, user_profile: UserProfile) -> str:
        """Provide savings strategy guidance"""
        if self.generator is not None:
            context = f"Savings goals and budgeting for {user_profile.user_type} with ${user_profile.income:,.2f} income"
            prompt = self.create_financial_prompt(query, user_profile, context)
            return self.generate_response(prompt)
        else:
            return self.get_fallback_response(query, user_profile, "savings")

# Global chatbot instance
chatbot = None

def initialize_chatbot(hf_token: str):
    """Initialize the chatbot with HuggingFace token"""
    global chatbot
    if not hf_token:
        return "❌ Please enter your HuggingFace token to initialize the chatbot."

    try:
        chatbot = FinanceChatbot(hf_token)
        if chatbot.generator is not None:
            return "✅ IBM Granite Finance Chatbot initialized successfully with AI model! All features available."
        else:
            return "⚠️ Chatbot initialized with fallback system. Basic features available. Check your token and model access."
    except Exception as e:
        return f"❌ Error initializing chatbot: {str(e)}"

def create_user_profile(name: str, age: int, occupation: str, income: float,
                       user_type: str, risk_tolerance: str, goals: str) -> str:
    """Create or update user profile"""
    if not chatbot:
        return "❌ Please initialize the chatbot first."

    if not name.strip():
        return "❌ Please enter a valid name."

    goal_list = [goal.strip() for goal in goals.split(',') if goal.strip()] if goals else []

    profile = UserProfile(
        name=name,
        age=age,
        occupation=occupation,
        income=income,
        user_type=user_type,
        risk_tolerance=risk_tolerance,
        financial_goals=goal_list
    )

    chatbot.user_profiles[name] = profile

    return f"""✅ Profile created for {name}!

**Profile Summary:**
- Type: {user_type.title()}
- Age: {age}
- Occupation: {occupation}
- Monthly Income: ${income:,.2f}
- Risk Tolerance: {risk_tolerance.title()}
- Goals: {', '.join(goal_list) if goal_list else 'Building financial wellness'}

You can now chat and use the budget analyzer!"""

def chat_interface(message: str, history: list, user_name: str) -> Tuple[str, list]:
    """Main chat interface"""
    if not chatbot:
        response = "❌ Please initialize the chatbot first."
        history.append([message, response])
        return response, history

    if not user_name or user_name not in chatbot.user_profiles:
        response = "❌ Please create a user profile first using the Setup tab."
        history.append([message, response])
        return response, history

    if not message.strip():
        response = "Please enter a financial question!"
        history.append([message, response])
        return response, history

    user_profile = chatbot.user_profiles[user_name]

    # Determine query type and route accordingly
    message_lower = message.lower()

    try:
        if any(word in message_lower for word in ['invest', 'stock', 'bond', 'portfolio', 'mutual fund', 'etf']):
            response = chatbot.get_investment_advice(message, user_profile)
        elif any(word in message_lower for word in ['tax', 'deduction', 'irs', 'refund', 'filing']):
            response = chatbot.get_tax_advice(message, user_profile)
        elif any(word in message_lower for word in ['save', 'saving', 'emergency fund', 'budget']):
            response = chatbot.get_savings_advice(message, user_profile)
        else:
            # General financial advice
            if chatbot.generator is not None:
                prompt = chatbot.create_financial_prompt(message, user_profile)
                response = chatbot.generate_response(prompt)
            else:
                response = chatbot.get_fallback_response(message, user_profile)

        # Update history
        history.append([message, response])
        return response, history

    except Exception as e:
        error_response = f"⚠️ I encountered an error: {str(e)}. Please try rephrasing your question."
        history.append([message, error_response])
        return error_response, history

def generate_budget_analysis(user_name: str, income: float,
                           housing: float, utilities: float, groceries: float,
                           transportation: float, entertainment: float,
                           dining_out: float, shopping: float, other: float) -> str:
    """Generate comprehensive budget analysis"""
    if not chatbot or not user_name or user_name not in chatbot.user_profiles:
        return "❌ Please initialize the chatbot and create a user profile first."

    if income <= 0:
        return "❌ Please enter a valid monthly income greater than $0."

    expenses = {
        "housing": housing,
        "utilities": utilities,
        "groceries": groceries,
        "transportation": transportation,
        "entertainment": entertainment,
        "dining_out": dining_out,
        "shopping": shopping,
        "other": other
    }

    user_profile = chatbot.user_profiles[user_name]

    try:
        budget_summary = chatbot.generate_budget_summary(income, expenses, user_profile)

        # Add basic analysis
        analysis = chatbot.analyze_spending_patterns(expenses, income)

        detailed_analysis = f"""## 💰 AI Budget Analysis for {user_name}

{budget_summary}

---

## 📊 Detailed Breakdown

**Monthly Snapshot:**
- 💵 Income: ${income:,.2f}
- 💸 Total Expenses: ${analysis['total_expenses']:,.2f}
- 💰 Available: ${income - analysis['total_expenses']:,.2f}
- 📈 Savings Rate: {analysis['savings_rate']:.1f}%

**Expense Categories:**
- 🏠 Housing: ${expenses['housing']:,.2f} ({expenses['housing']/income*100:.1f}%)
- ⚡ Utilities: ${expenses['utilities']:,.2f} ({expenses['utilities']/income*100:.1f}%)
- 🛒 Groceries: ${expenses['groceries']:,.2f} ({expenses['groceries']/income*100:.1f}%)
- 🚗 Transportation: ${expenses['transportation']:,.2f} ({expenses['transportation']/income*100:.1f}%)
- 🎬 Entertainment: ${expenses['entertainment']:,.2f} ({expenses['entertainment']/income*100:.1f}%)
- 🍽️ Dining Out: ${expenses['dining_out']:,.2f} ({expenses['dining_out']/income*100:.1f}%)
- 🛍️ Shopping: ${expenses['shopping']:,.2f} ({expenses['shopping']/income*100:.1f}%)
- 📝 Other: ${expenses['other']:,.2f} ({expenses['other']/income*100:.1f}%)

---

**💡 Key Insights:**
{chr(10).join(f'• {insight}' for insight in analysis['insights']) if analysis['insights'] else '• Your budget looks balanced!'}

**🎯 Action Items:**
{chr(10).join(f'• {rec}' for rec in analysis['recommendations']) if analysis['recommendations'] else '• Keep up the great work!'}
        """

        return detailed_analysis.strip()

    except Exception as e:
        return f"❌ Error generating budget analysis: {str(e)}"

# Create Gradio Interface
def create_interface():
    with gr.Blocks(title="Personal Finance Chatbot - IBM Granite AI", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🏦 Personal Finance Chatbot
        ### Powered by IBM Granite 3.3 AI Model

        Get personalized financial guidance on savings, taxes, investments, and budgeting!

        ---
        """)

        with gr.Tab("🔧 Setup & Configuration"):
            gr.Markdown("## Step 1: Initialize Chatbot")
            with gr.Row():
                hf_token_input = gr.Textbox(
                    label="🔑 HuggingFace Token",
                    type="password",
                    placeholder="Enter your HuggingFace token (get from: https://huggingface.co/settings/tokens)",
                    info="Required to access IBM Granite model - create a free account at HuggingFace"
                )
                init_btn = gr.Button("🚀 Initialize Chatbot", variant="primary", size="lg")

            init_status = gr.Markdown("Status: Waiting for initialization...")

            gr.Markdown("---")
            gr.Markdown("## Step 2: Create Your Financial Profile")
            with gr.Row():
                with gr.Column():
                    profile_name = gr.Textbox(label="👤 Name", placeholder="Enter your name")
                    profile_age = gr.Number(label="🎂 Age", value=25, minimum=16, maximum=100)
                    profile_occupation = gr.Textbox(label="💼 Occupation", placeholder="e.g., Student, Engineer, Teacher")
                    profile_income = gr.Number(label="💰 Monthly Income ($)", value=5000, minimum=0)

                with gr.Column():
                    profile_type = gr.Radio(
                        label="👥 User Type",
                        choices=["student", "professional"],
                        value="professional",
                        info="This affects the complexity and style of advice"
                    )
                    profile_risk = gr.Radio(
                        label="📊 Risk Tolerance",
                        choices=["conservative", "moderate", "aggressive"],
                        value="moderate",
                        info="For investment recommendations"
                    )
                    profile_goals = gr.Textbox(
                        label="🎯 Financial Goals",
                        placeholder="e.g., Emergency fund, House down payment, Retirement",
                        info="Separate multiple goals with commas",
                        lines=2
                    )

            create_profile_btn = gr.Button("✅ Create Profile", variant="primary", size="lg")
            profile_status = gr.Markdown("Status: No profile created yet")

        with gr.Tab("💬 Financial Chat Assistant"):
            gr.Markdown("### 🤖 Ask your financial questions!")

            with gr.Row():
                chat_user_name = gr.Textbox(
                    label="👤 Your Name",
                    placeholder="Enter the name from your profile",
                    info="Must match the name in your profile"
                )

            chatbot_interface = gr.Chatbot(
                height=500,
                placeholder="Start chatting by entering your name above and asking a financial question!",
                avatar_images=["👤", "🏦"]
            )

            msg = gr.Textbox(
                label="💭 Your Message",
                placeholder="Ask about investments, taxes, savings, or any financial topic...",
                lines=3,
                max_lines=5
            )

            with gr.Row():
                send_btn = gr.Button("📤 Send", variant="primary")
                clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary")

            gr.Markdown("""
            ### 💡 Example Questions to Try:

            **💰 Investment Questions:**
            - "How should I start investing as a beginner?"
            - "What's the best asset allocation for my age?"
            - "Should I invest in individual stocks or index funds?"

            **📊 Tax Questions:**
            - "What tax deductions can I claim?"
            - "Should I use a Roth or traditional IRA?"
            - "How can I minimize my tax burden?"

            **💳 Budgeting & Savings:**
            - "How much should I save for emergencies?"
            - "What's the best budgeting strategy for my situation?"
            - "How can I improve my spending habits?"
            """)

        with gr.Tab("📊 Budget Analyzer"):
            gr.Markdown("### 🔍 AI-Powered Budget Analysis")

            budget_user_name = gr.Textbox(
                label="👤 Your Name",
                placeholder="Enter the name from your profile"
            )

            gr.Markdown("#### 💵 Enter Your Monthly Finances")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("**💰 Income**")
                    budget_income = gr.Number(label="Monthly Income ($)", value=5000, minimum=0)

                    gr.Markdown("**🏠 Essential Expenses**")
                    budget_housing = gr.Number(label="Housing/Rent ($)", value=1500)
                    budget_utilities = gr.Number(label="Utilities ($)", value=200)
                    budget_groceries = gr.Number(label="Groceries ($)", value=400)
                    budget_transportation = gr.Number(label="Transportation ($)", value=300)

                with gr.Column(scale=1):
                    gr.Markdown("**🎯 Discretionary Expenses**")
                    budget_entertainment = gr.Number(label="Entertainment ($)", value=200)
                    budget_dining = gr.Number(label="Dining Out ($)", value=300)
                    budget_shopping = gr.Number(label="Shopping ($)", value=250)
                    budget_other = gr.Number(label="Other Expenses ($)", value=150)

            analyze_btn = gr.Button("📈 Generate AI Budget Analysis", variant="primary", size="lg")
            budget_result = gr.Markdown("Your budget analysis will appear here...")

            gr.Markdown("""
            ### 📋 Budget Tips:
            - **50/30/20 Rule**: 50% needs, 30% wants, 20% savings
            - **Emergency Fund**: Save 3-6 months of expenses
            - **Track Everything**: Small expenses add up quickly
            - **Review Monthly**: Adjust your budget as needed
            """)

        with gr.Tab("📚 Financial Education"):
            gr.Markdown("""
            # 📖 Financial Education Center

            ## 🎯 Core Financial Concepts

            ### 💰 Investment Basics
            - **Stocks**: Ownership shares in companies
            - **Bonds**: Loans to governments/companies that pay interest
            - **Mutual Funds**: Professionally managed, diversified portfolios
            - **ETFs**: Low-cost funds that track market indices
            - **Index Funds**: Passive investing in market benchmarks

            ### 📊 Risk & Return
            - **Conservative**: Lower risk, lower returns (bonds, savings)
            - **Moderate**: Balanced risk/return (60/40 stocks/bonds)
            - **Aggressive**: Higher risk, higher potential returns (growth stocks)

            ### 💳 Budgeting Strategies
            - **50/30/20 Rule**: 50% needs, 30% wants, 20% savings
            - **Zero-Based**: Every dollar has a purpose
            - **Pay Yourself First**: Save before spending
            - **Envelope Method**: Cash allocation for categories

            ### 🏦 Account Types
            - **401(k)**: Employer-sponsored retirement account
            - **Traditional IRA**: Tax-deferred retirement savings
            - **Roth IRA**: Tax-free growth and withdrawals
            - **HSA**: Triple tax advantage for medical expenses

            ### 📈 Key Financial Metrics
            - **Net Worth**: Assets - Liabilities
            - **Debt-to-Income**: Monthly debt payments / Monthly income
            - **Savings Rate**: Monthly savings / Monthly income
            - **Emergency Fund**: 3-6 months of expenses

            ## 🎓 Learning Resources
            - **Books**: "A Random Walk Down Wall Street", "The Bogleheads' Guide"
            - **Podcasts**: "The Investors Podcast", "Planet Money"
            - **Websites**: Investopedia, Morningstar, SEC.gov
            - **Tools**: Personal Capital, Mint, YNAB for budgeting
            """)

        # Event handlers
        init_btn.click(
            initialize_chatbot,
            inputs=[hf_token_input],
            outputs=[init_status]
        )

        create_profile_btn.click(
            create_user_profile,
            inputs=[profile_name, profile_age, profile_occupation, profile_income,
                   profile_type, profile_risk, profile_goals],
            outputs=[profile_status]
        )

        send_btn.click(
            chat_interface,
            inputs=[msg, chatbot_interface, chat_user_name],
            outputs=[msg, chatbot_interface]
        ).then(lambda: "", outputs=[msg])

        msg.submit(
            chat_interface,
            inputs=[msg, chatbot_interface, chat_user_name],
            outputs=[msg, chatbot_interface]
        ).then(lambda: "", outputs=[msg])

        clear_btn.click(lambda: [], outputs=[chatbot_interface])

        analyze_btn.click(
            generate_budget_analysis,
            inputs=[budget_user_name, budget_income, budget_housing, budget_utilities,
                   budget_groceries, budget_transportation, budget_entertainment,
                   budget_dining, budget_shopping, budget_other],
            outputs=[budget_result]
        )

    return demo

# Main execution
if __name__ == "__main__":
    print("🚀 Starting Personal Finance Chatbot...")
    print("📦 If this is your first time, packages will be installed automatically...")

    try:
        demo = create_interface()
        print("✅ Interface created successfully!")
        print("🌐 Launching web interface...")

        demo.launch(
            server_name="0.0.0.0",
            server_port=7860,
            share=False,
            show_error=True,
            quiet=False
        )

    except Exception as e:
        print(f"❌ Error launching application: {e}")
        print("\n🔧 Troubleshooting Steps:")
        print("1. Restart your Python kernel/runtime")
        print("2. Run: pip install gradio transformers torch pandas accelerate huggingface-hub")
        print("3. Make sure you have a valid HuggingFace token")
        print("4. Check your internet connection")

print("🎉 Personal Finance Chatbot setup complete!")
print("📋 Next steps:")
print("1. Run the code above")
print("2. Get your HuggingFace token from: https://huggingface.co/settings/tokens")
print("3. Initialize the chatbot with your token")
print("4. Create your financial profile")
print("5. Start asking financial questions!")

📦 Installing gradio...
✅ transformers>=4.35.0 already installed
✅ torch already installed
✅ pandas already installed


/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


✅ accelerate already installed
📦 Installing huggingface-hub...
✅ numpy already installed
🎉 All packages installed successfully!
🚀 Starting Personal Finance Chatbot...
📦 If this is your first time, packages will be installed automatically...
✅ Interface created successfully!
🌐 Launching web interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

🎉 Personal Finance Chatbot setup complete!
📋 Next steps:
1. Run the code above
2. Get your HuggingFace token from: https://huggingface.co/settings/tokens
3. Initialize the chatbot with your token
4. Create your financial profile
5. Start asking financial questions!
